# Count-4: claim count >= 11 or claim count = 0 (flag)

Description: the test is to check if all the claim counts are greater or equal to 11, expect for zero. Zero claim counts are flagged.  

Starting Author: Amy Jin (amy@careset.com)

Date: April 16th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [5]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) col_name:               column to test
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, the test will print out the failed claim counts, i.e., the claim counts < 11 
#    and claim counts not equal to 0.

def count_4(db_name, table_name, col_name):

    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        query = ('''
            SELECT {col}
            FROM {db}.{t1}
            WHERE {col} <>0 AND {col} < 11
            LIMIT 1;
        '''.format(col = col_name, db = db_name, t1 = table_name))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))

        if not len(rows):
            print ("Count-1 test result for {}.{} : PASS".format(db_name, table_name))
        else:
                   
            print ("Count-1 test result for {}.{} : FAIL".format(db_name, table_name) + '\n')
            print ("The following {} fail the test:".format(col_name) + '\n')
        
            query = ('''
                SELECT {col}
                FROM {db}.{t1}
                WHERE {col} <>0 AND {col} < 11;
            '''.format(col = col_name, db = db_name, t1 = table_name))

            cur.execute(query)
                
            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]))
                print ('\n')
            
        cur.close()
        connection.close()            

## Test Example

In [6]:
count_4('_amy', 'test_data_good', 'cnt_clm_id')

Count-1 test result for _amy.test_data_good : PASS


In [7]:
count_4('_amy', 'test_data_bad1', 'cnt_clm_id')

Count-1 test result for _amy.test_data_bad1 : FAIL

The following cnt_clm_id fail the test:

5.0


9.0


2.0


7.0


7.0


6.0


7.0


9.0


10.0


4.0


4.0


6.0


8.0


7.0


6.0


10.0


3.0


1.0


9.0


8.0


4.0


10.0


7.0


2.0


3.0


10.0


6.0


1.0


6.0


3.0


7.0


9.0


5.0


9.0


8.0


8.0


3.0


3.0


1.0


9.0


4.0


5.0


5.0


6.0


7.0


9.0


10.0


6.0


4.0


6.0


3.0


3.0


10.0


5.0


4.0


2.0


1.0


3.0


4.0


10.0


5.0


8.0


6.0


3.0


4.0


3.0


5.0


6.0


2.0


4.0


7.0


9.0


3.0


5.0


8.0


3.0


9.0


10.0


4.0


10.0


7.0


4.0


2.0


10.0


5.0


9.0


3.0


6.0


4.0


2.0


5.0


4.0


10.0


1.0


3.0


10.0


3.0


6.0


10.0


5.0


6.0


10.0


9.0


4.0


1.0


8.0


7.0


3.0


8.0


4.0


8.0


5.0


1.0


8.0


7.0


2.0


8.0


9.0


3.0


4.0


9.0


6.0


3.0


9.0


1.0


7.0


9.0


7.0


3.0


8.0


7.0


6.0


3.0


4.0


4.0


9.0


8.0


9.0


7.0


7.0


9.0


7.0


8.0


9.0


7.0


2.0


9.0


9.0


4.0

